In [3]:
import pandas as pd
from gensim import corpora, models
from googlesearch import search
import re
import chardet

In [4]:
# Lets check what encoding we have for our NSFdata files
NSF_csv_files = ['NSFdata/NSF_CCF.csv', 'NSFdata/NSF_CICI.csv', 'NSFdata/NSF_CSSI.csv', 'NSFdata/NSF_DIBBS.csv', 'NSFdata/NSF_MRI.csv', 'NSFdata/NSF_OAC.csv', 'NSFdata/NSF_SI2.csv']
for file in NSF_csv_files:
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        print(file, result)

NSFdata/NSF_CCF.csv {'encoding': 'ISO-8859-1', 'confidence': 0.7299916171744574, 'language': ''}
NSFdata/NSF_CICI.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_CSSI.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_DIBBS.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_MRI.csv {'encoding': 'ISO-8859-1', 'confidence': 0.7299962504897843, 'language': ''}
NSFdata/NSF_OAC.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_SI2.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [5]:
# 1. Read the CSV file and load it into a DataFrame
data = pd.read_csv('NSFdata/NSF_DIBBS.csv', encoding='ISO-8859-1')

In [9]:
projects = data[["AwardNumber", "Title", "NSFOrganization", "PrincipalInvestigator", "PIEmailAddress", "Abstract"]]
print(projects["Abstract"])

0      This project would automate the creation and d...
1      The growing number of cyber attacks on the Int...
2      Uranium-series geochronology plays a critical ...
3      Cybersecurity has become a significant issue t...
4      CIF21 DIBBs: Conceptualization of the Social a...
                             ...                        
125    ABSTRACT<br/><br/>OPP-9813312   OPP-9813442   ...
126    ABSTRACT<br/><br/>OPP-9813312   OPP-9813442   ...
127    ABSTRACT<br/><br/>OPP-9907197    OPP-9907469  ...
128    Current general circulation models (GCMs) have...
129    ABSTRACT<br/><br/>OPP-9907197    OPP-9907469  ...
Name: Abstract, Length: 130, dtype: object


In [11]:
# 3. Process the Abstract column using LDA
abstracts = projects["Abstract"].apply(lambda x: re.sub('<[^<]+?>', '', x).split())  # Remove HTML tags and split into words
print(abstracts)